In [26]:
import pandas as pd
import torch
import torchvision.transforms as transforms
import pickle

from data_utils import get_karpathy_split
from data_loader import get_caption_loader, COCOCaptionDataset

from build_vocab import Vocabulary

In [10]:
with open('data/coco_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [3]:
caps_df = get_karpathy_split(splits_path='/home/simeon/Dokumente/Code/Data/COCO/splits/karpathy/caption_datasets/', caps_path='/home/simeon/Dokumente/Code/Data/COCO/')

In [8]:
crop_size=224
image_dir='/home/simeon/Dokumente/Code/Data/COCO/'

In [12]:
transform = transforms.Compose([
    transforms.Resize((crop_size, crop_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

In [13]:
loader = get_caption_loader(
        decoding_level='word', 
        split='train',
        data_df=caps_df, 
        image_dir=image_dir, 
        vocab=vocab,
        transform=transform, 
        batch_size=20, 
        shuffle=False,
        num_workers=2, 
        drop_last=False
)

In [31]:
eval_dset = COCOCaptionDataset(
    split='val', 
    data_df=caps_df.groupby('image_id').agg('first').reset_index(), 
    image_dir='/home/simeon/Dokumente/Code/Data/COCO/',
    vocab=vocab, 
    decoding_level='word', 
    transform=None
)

In [47]:
pd.unique(caps_df.split)

array(['train', 'restval', 'test', 'val'], dtype=object)

In [37]:
for i, (image, target, target_len, img_id, img_path) in enumerate(eval_dset):
    target = target.long()
    if i > 1:
        break

In [17]:
for i, (images, captions, lengths, _, _) in enumerate(loader):    
    if i > 2:
        break
    print(i)

0
1
2


In [25]:
idx = [i.item() for i in captions[2]]
' '.join([vocab.idx2word[i] for i in idx])

'<start> the top of a kitchen cabinet covered with brass pots and pans <end> <pad> <pad> <pad>'